In [5]:
# Import necessary modules
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64
from pymongo import MongoClient
import os

# CRUD class for MongoDB interactions
class CRUD:
    def __init__(self, username, password, host, port, db_name, collection_name):
        self.client = MongoClient(f"mongodb://{username}:{password}@{host}:{port}/?authSource=admin")
        self.db = self.client[db_name]
        self.collection = self.db[collection_name]

    def read(self, query):
        try:
            result = list(self.collection.find(query))
            return result
        except Exception as e:
            print(f"An error occurred: {e}")
            return []

# Replace with correct credentials
crud = CRUD(
    "root",                        
    "zZT91rXhbs",                  
    "nv-desktop-services.apporto.com",  
    30018,                         
    "animal_rescue_db",            
    "dogs"                         
)

# Load data from MongoDB using the read function
df = pd.DataFrame.from_records(crud.read({}))

# Remove MongoDB _id field if it exists
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Ensure longitude and latitude are numeric
df['location_long'] = pd.to_numeric(df['location_long'], errors='coerce')
df['location_lat'] = pd.to_numeric(df['location_lat'], errors='coerce')

# Define rescue type coordinates (replace with real data)
rescue_coordinates = {
    'Water Rescue': {'lat_min': 30.25, 'lat_max': 30.30, 'long_min': -97.75, 'long_max': -97.70},
    'Mountain/Wilderness Rescue': {'lat_min': 30.28, 'lat_max': 30.32, 'long_min': -97.78, 'long_max': -97.74},
    'Disaster Rescue': {'lat_min': 30.20, 'lat_max': 30.25, 'long_min': -97.80, 'long_max': -97.76},
    'Individual Tracking': {'lat_min': 30.29, 'lat_max': 30.33, 'long_min': -97.73, 'long_max': -97.69}
}

# Initialize the Dash app
app = JupyterDash(__name__)

# Load and encode the Grazioso Salvare logo
image_path = '/home/serenityroger_snhu/Desktop/Grazioso Salvare Logo.png'
image_src = ''
try:
    encoded_image = base64.b64encode(open(image_path, 'rb').read())
    image_src = f"data:image/png;base64,{encoded_image.decode()}"
except FileNotFoundError:
    pass

####################
# Dashboard Layout
####################
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    
    # Logo Image
    html.Img(src=image_src, style={'width': '200px'}) if image_src else html.Div(),
    
    html.Hr(),
    html.H3("Presented by Serenity Rogers"),  # Add name identifier
    
    # Filter Options for Rescue Types
    html.Div([
        html.Label('Filter by Rescue Type:'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'all'},
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain/Wilderness Rescue'},
                {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
                {'label': 'Individual Tracking', 'value': 'Individual Tracking'}
            ],
            value='all',
            labelStyle={'display': 'inline-block'}
        ),
    ], style={'margin': '20px 0'}),
    
    html.Hr(),
    
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        filter_action='native',
        sort_action='native',
        page_size=10
    ),
    
    html.Br(),
    html.Hr(),
    
    # Graph and Map Divs
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

############################
# Callbacks and Interaction
############################

# Callback to filter the data based on Rescue Type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'all':
        return df.to_dict('records')
    elif filter_type in rescue_coordinates:
        # Filter by latitude and longitude based on rescue type
        coords = rescue_coordinates[filter_type]
        filtered_df = df[
            (df['location_lat'].between(coords['lat_min'], coords['lat_max'])) &
            (df['location_long'].between(coords['long_min'], coords['long_max']))
        ]
        return filtered_df.to_dict('records')

# Callback for graph update based on filtered data
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return html.Div("No data available for the selected filter.")
    
    dff = pd.DataFrame(viewData)
    
    if 'breed' in dff.columns:
        # Example Pie Chart
        return [
            dcc.Graph(
                figure=px.pie(dff, names='breed', title='Preferred Animal Breeds')
            )
        ]
    else:
        return html.Div("Breed data not available.")

# Callback for map update based on filtered data and table selection
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    if viewData is None or len(viewData) == 0:
        return html.Div("No map data available.")
    
    dff = pd.DataFrame(viewData)
    
    if not index:
        # If no rows are selected, show all filtered rescue locations
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff['location_lat'].mean(), dff['location_long'].mean()], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.LayerGroup([dl.Marker(position=[row['location_lat'], row['location_long']], children=[
                dl.Tooltip(row['name']),  # Add tooltip with name
                dl.Popup([html.H1("Rescue Info"), html.P(row.get('outcome_subtype', 'No description available'))])
            ]) for _, row in dff.iterrows() if not pd.isna(row['location_lat']) and not pd.isna(row['location_long'])])
        ])
    
    # Zoom in on selected row
    row = dff.iloc[index[0]]
    return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[row['location_lat'], row['location_long']], zoom=12, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[row['location_lat'], row['location_long']], children=[
            dl.Tooltip(row['name']),
            dl.Popup([html.H1("Rescue Info"), html.P(row.get('outcome_subtype', 'No description available'))])
        ])
    ])

# Run the app
app.run_server(debug=True)



Dash app running on http://127.0.0.1:11275/
